In [ ]:
import pandas as pd
import numpy as np
import sys
import os

# Add the src directory to Python path
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from mimiciii_db import DB
from mimiciii_db.config import db_url

In [ ]:
db = DB.from_url(db_url())
print("Database connected successfully!")

In [ ]:
lca = pd.read_csv('/Users/gloriaye/Desktop/dsc180ab/25fa-dsc180a-team1/data/lca_all_subgroups_relabeled.csv')

In [ ]:
lca

In [ ]:
filtered_patients_with_morbidity_counts = db.table_df("filtered_patients_with_morbidity_counts", schema="mimiciii")
filtered_patients_with_morbidity_counts

In [ ]:

# --- Load
sofa_df  = pd.read_csv('/Users/gloriaye/Desktop/dsc180ab/25fa-dsc180a-team1/csv/sofa.csv')
oasis_df = pd.read_csv('/Users/gloriaye/Desktop/dsc180ab/25fa-dsc180a-team1/csv/oasis.csv')

patients = filtered_patients_with_morbidity_counts.copy()

# --- Ensure key dtypes match
keys = ['subject_id','hadm_id']
for df in (patients, sofa_df, oasis_df):
    for k in keys:
        df[k] = pd.to_numeric(df[k], errors='coerce')

# --- (Optional) de-duplicate / aggregate if needed
# keep first; or replace with .groupby(keys).agg({'sofa_total':'max', ...})
sofa_df  = sofa_df.drop_duplicates(subset=keys)
oasis_df = oasis_df.drop_duplicates(subset=keys)

# --- Left join SOFA then OASIS
merged = (
    patients
      .merge(sofa_df,  on=keys, how='left', suffixes=('', '_sofa'))
      .merge(oasis_df, on=keys, how='left', suffixes=('', '_oasis'))
)

print("patients rows:", len(patients))
print("merged rows  :", len(merged))        # should be the same
print("new columns  :", [c for c in merged.columns if c not in patients.columns])

# --- Save
out = '/Users/gloriaye/Desktop/dsc180ab/25fa-dsc180a-team1/csv/filtered_patients_with_morbidity_counts_sofa_oasis.csv'
merged.to_csv(out, index=False)
print("✓ Saved to:", out)


In [ ]:
merged

In [ ]:
merged.columns

In [ ]:
df_merged = merged.merge(lca[['hadm_id', 'subgroup_K6']],
                            on='hadm_id', how='left')


In [ ]:
df_merged['subgroup_K6'] = pd.to_numeric(df_merged['subgroup_K6'], errors='coerce').astype('Int64')
df_merged

In [ ]:
df_merged.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

order  = [1,2,3,4,5,6]
colors = ['white', '#E74C3C', '#27AE60', '#1F77B4', '#17BECF', '#E377C2']  # 1–6

def colored_boxplot(ax, ycol):
    data = [df_merged.loc[df_merged['subgroup_K6']==k, ycol].dropna() for k in order]
    bp = ax.boxplot(data, labels=order, showfliers=True, patch_artist=True)
    for patch, c in zip(bp['boxes'], colors):
        patch.set_facecolor(c)
        patch.set_edgecolor('black')
    for w in bp['whiskers'] + bp['caps']:
        w.set_color('black')
    for m in bp['medians']:
        m.set_color('black'); m.set_linewidth(2)
    ax.set_xlabel('Subgroup')

fig, axes = plt.subplots(1, 2, figsize=(10,4))
colored_boxplot(axes[0], 'sofa');  axes[0].set_title('A'); axes[0].set_ylabel('SOFA score')
colored_boxplot(axes[1], 'oasis'); axes[1].set_title('B'); axes[1].set_ylabel('OASIS score')
plt.tight_layout()

# Save figure
import os
os.makedirs('/Users/gloriaye/Desktop/dsc180ab/25fa-dsc180a-team1/assets', exist_ok=True)
plt.savefig('/Users/gloriaye/Desktop/dsc180ab/25fa-dsc180a-team1/assets/fig_4_ab.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
query = f"""-- THIS SCRIPT IS AUTOMATICALLY GENERATED. DO NOT EDIT IT DIRECTLY.
DROP TABLE IF EXISTS angus; CREATE TABLE angus AS 
-- ICD-9 codes for Angus criteria of sepsis

-- Angus et al, 2001. Epidemiology of severe sepsis in the United States
-- http://www.ncbi.nlm.nih.gov/pubmed/11445675

-- Case selection and definitions
-- To identify cases with severe sepsis, we selected all acute care
-- hospitalizations with ICD-9-CM codes for both:
-- (a) a bacterial or fungal infectious process AND
-- (b) a diagnosis of acute organ dysfunction (Appendix 2).

-- ICD-9 codes for infection - as sourced from Appendix 1 of above paper
WITH infection_group AS
(
	SELECT subject_id, hadm_id,
	CASE
		WHEN SUBSTR(icd9_code,1,3) IN ('001','002','003','004','005','008',
			   '009','010','011','012','013','014','015','016','017','018',
			   '020','021','022','023','024','025','026','027','030','031',
			   '032','033','034','035','036','037','038','039','040','041',
			   '090','091','092','093','094','095','096','097','098','100',
			   '101','102','103','104','110','111','112','114','115','116',
			   '117','118','320','322','324','325','420','421','451','461',
			   '462','463','464','465','481','482','485','486','494','510',
			   '513','540','541','542','566','567','590','597','601','614',
			   '615','616','681','682','683','686','730') THEN 1
		WHEN SUBSTR(icd9_code,1,4) IN ('5695','5720','5721','5750','5990','7110',
				'7907','9966','9985','9993') THEN 1
		WHEN SUBSTR(icd9_code,1,5) IN ('49121','56201','56203','56211','56213',
				'56983') THEN 1
		ELSE 0 END AS infection
	from diagnoses_icd
),
-- ICD-9 codes for organ dysfunction - as sourced from Appendix 2 of above paper
organ_diag_group as
(
	SELECT subject_id, hadm_id,
		CASE
		-- Acute Organ Dysfunction Diagnosis Codes
		WHEN SUBSTR(icd9_code,1,3) IN ('458','293','570','584') THEN 1
		WHEN SUBSTR(icd9_code,1,4) IN ('7855','3483','3481',
				'2874','2875','2869','2866','5734')  THEN 1
		ELSE 0 END AS organ_dysfunction,
		-- Explicit diagnosis of severe sepsis or septic shock
		CASE
		WHEN SUBSTR(icd9_code,1,5) IN ('99592','78552')  THEN 1
		ELSE 0 END AS explicit_sepsis
	from diagnoses_icd
),
-- Mechanical ventilation
organ_proc_group as
(
	SELECT subject_id, hadm_id,
		CASE
		WHEN icd9_code IN ('9670', '9671', '9672') THEN 1
		ELSE 0 END AS mech_vent
	FROM procedures_icd
),
-- Aggregate above views together
aggregate as
(
	SELECT subject_id, hadm_id,
		CASE
			WHEN hadm_id in
					(SELECT DISTINCT hadm_id
					FROM infection_group
					WHERE infection = 1)
				THEN 1
			ELSE 0 END AS infection,
		CASE
			WHEN hadm_id in
					(SELECT DISTINCT hadm_id
					FROM organ_diag_group
					WHERE explicit_sepsis = 1)
				THEN 1
			ELSE 0 END AS explicit_sepsis,
		CASE
			WHEN hadm_id in
					(SELECT DISTINCT hadm_id
					FROM organ_diag_group
					WHERE organ_dysfunction = 1)
				THEN 1
			ELSE 0 END AS organ_dysfunction,
		CASE
		WHEN hadm_id in
				(SELECT DISTINCT hadm_id
				FROM organ_proc_group
				WHERE mech_vent = 1)
			THEN 1
		ELSE 0 END AS mech_vent
	FROM admissions
)
-- Output component flags (explicit sepsis, organ dysfunction) and final flag (angus)
SELECT subject_id, hadm_id, infection,
   explicit_sepsis, organ_dysfunction, mech_vent,
CASE
	WHEN explicit_sepsis = 1 THEN 1
	WHEN infection = 1 AND organ_dysfunction = 1 THEN 1
	WHEN infection = 1 AND mech_vent = 1 THEN 1
	ELSE 0 END
AS angus
FROM aggregate;"""

In [ ]:
# Execute the CREATE TABLE statement first (DDL)
db.execute(query)

# Then query the created table
angus = db.query_df("SELECT * FROM angus")

In [ ]:
angus

In [ ]:
admin = angus.copy()   # <- replace with the DataFrame you showed (the one in your screenshot)
keys = ['subject_id','hadm_id']

for df_ in (df_merged, admin):
    for k in keys:
        df_[k] = pd.to_numeric(df_[k], errors='coerce')

admin = admin.drop_duplicates(subset=keys)           # 1 row per (subject, hadm)
dfA = df_merged.merge(admin, on=keys, how='left')


In [ ]:
dfA

In [ ]:
import numpy as np
import pandas as pd

# Create sepsis column from angus if it doesn't exist
if 'sepsis' not in dfA.columns and 'angus' in dfA.columns:
    dfA['sepsis'] = dfA['angus']

# Create mortality column from hospital_expire_flag if it doesn't exist
if 'mortality' not in dfA.columns and 'hospital_expire_flag' in dfA.columns:
    dfA['mortality'] = dfA['hospital_expire_flag']

# make sure 0/1 and subgroup are numeric
for c in ['organ_dysfunction','sepsis','mortality']:
    dfA[c] = pd.to_numeric(dfA[c], errors='coerce')

dfA['subgroup_K6'] = pd.to_numeric(dfA['subgroup_K6'], errors='coerce').astype('Int64')
dfA = dfA[dfA['subgroup_K6'].between(1,6)].copy()

by = dfA.groupby('subgroup_K6', dropna=True)
n = by.size()

# proportions (%)
organ_p = by['organ_dysfunction'].mean()
sepsis_p = by['sepsis'].mean()
mort_p   = by['mortality'].mean()

# binomial SEs (in % points)
organ_se = np.sqrt(organ_p*(1-organ_p)/n) * 100
sepsis_se = np.sqrt(sepsis_p*(1-sepsis_p)/n) * 100
mort_se   = np.sqrt(mort_p*(1-mort_p)/n) * 100

stats = pd.DataFrame({
    'n': n.astype(int),
    'organ_pct': 100*organ_p,
    'organ_se':  organ_se,
    'sepsis_pct': 100*sepsis_p,
    'sepsis_se':  sepsis_se,
    'mort_pct':  100*mort_p,
    'mort_se':   mort_se
}).reset_index().sort_values('subgroup_K6')

print(stats.round(2))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ----- 1) clean inputs -----
df = dfA.copy()
df = df.drop_duplicates(subset=['subject_id','hadm_id'])  # 1 row per admission
df['subgroup_K6'] = pd.to_numeric(df['subgroup_K6'], errors='coerce').astype('Int64')
df = df[df['subgroup_K6'].between(1,6)].copy()
for c in ['organ_dysfunction','sepsis','mortality']:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(int)

# ----- 2) helpers -----
def wilson_ci(k, n, z=1.96):
    if n == 0: return np.nan, np.nan
    p = k / n
    den = 1 + z**2/n
    cen = (p + z**2/(2*n)) / den
    half = z*np.sqrt((p*(1-p) + z**2/(4*n))/n) / den
    return cen-half, cen+half

def prop_table(flag):
    g = df.groupby('subgroup_K6')[flag]
    n = g.size()
    k = g.sum()
    p = k / n
    lo, hi = zip(*[wilson_ci(int(kk), int(nn)) for kk, nn in zip(k, n)])
    out = pd.DataFrame({
        'subgroup': n.index.astype(int),
        'n': n.values,
        'p': 100*p.values,
        'lo': 100*np.array(lo),
        'hi': 100*np.array(hi),
    }).sort_values('subgroup')
    out['err_low']  = out['p'] - out['lo']
    out['err_high'] = out['hi'] - out['p']
    return out

organ_stats = prop_table('organ_dysfunction')
sepsis_stats = prop_table('sepsis')
mort_stats   = prop_table('mortality')

# ----- 3) colors (match paper’s subgroup palette) -----
# 1:white, 2:red, 3:green, 4:blue, 5:cyan, 6:magenta
colors = {1:'#FFFFFF', 2:'#E41A1C', 3:'#4DAF4A', 4:'#377EB8', 5:'#4DD2D2', 6:'#E377C2'}

# ----- 4) Panel C: prevalence (organ dysfunction & sepsis) -----
x = organ_stats['subgroup'].to_numpy()
w = 0.32

fig, ax = plt.subplots(figsize=(6,4))
for i, sg in enumerate(x):
    col = colors[sg]
    # left: organ dysfunction
    ax.bar(sg - w/2, organ_stats.loc[organ_stats['subgroup']==sg, 'p'],
           width=w, yerr=np.vstack([organ_stats.loc[organ_stats['subgroup']==sg, 'err_low'],
                                    organ_stats.loc[organ_stats['subgroup']==sg, 'err_high']]),
           capsize=3, edgecolor='black', linewidth=1.0, color=col)
    # right: sepsis
    ax.bar(sg + w/2, sepsis_stats.loc[sepsis_stats['subgroup']==sg, 'p'],
           width=w, yerr=np.vstack([sepsis_stats.loc[sepsis_stats['subgroup']==sg, 'err_low'],
                                    sepsis_stats.loc[sepsis_stats['subgroup']==sg, 'err_high']]),
           capsize=3, edgecolor='black', linewidth=1.0, color=col)

ax.set_xlabel('Subgroup (K=6)')
ax.set_ylabel('Percent prevalence')
ax.set_xticks(x)
ax.set_ylim(0, 80)             # match paper
ax.set_title('C')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(True, linestyle=':', linewidth=0.7, alpha=0.6)
plt.tight_layout()
plt.show()
plt.savefig('/Users/gloriaye/Desktop/dsc180ab/25fa-dsc180a-team1/assets/fig_4_c.png', dpi=300, bbox_inches='tight')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = dfA.drop_duplicates(['subject_id','hadm_id']).copy()
df['subgroup_K6'] = pd.to_numeric(df['subgroup_K6'], errors='coerce').astype('Int64')
df = df[df['subgroup_K6'].between(1,6)].copy()

# ensure 0/1 ints
for c in ['organ_dysfunction','sepsis','mortality']:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(int)

# Wilson CI for proportions
def wilson_ci(k, n, z=1.96):
    if n == 0: return (np.nan, np.nan)
    p = k/n
    den = 1 + z**2/n
    cen = (p + z**2/(2*n))/den
    half = z*np.sqrt((p*(1-p) + z**2/(4*n))/n)/den
    return cen-half, cen+half

def conditional_mort(flag_col):
    # compute mortality ONLY among rows with flag==1, by subgroup
    groups = []
    for sg, sub in df.groupby('subgroup_K6'):
        sub = sub[sub[flag_col] == 1]
        n = len(sub)
        k = sub['mortality'].sum()
        lo, hi = wilson_ci(k, n)
        groups.append({'subgroup': int(sg),
                       'n': n,
                       'p': 100*(k/n) if n>0 else np.nan,
                       'err_low': 100*((k/n)-lo) if n>0 else np.nan,
                       'err_high': 100*(hi-(k/n)) if n>0 else np.nan})
    out = pd.DataFrame(groups).sort_values('subgroup')
    return out

mort_od  = conditional_mort('organ_dysfunction')   # mortality among organ-dysfunction
mort_sep = conditional_mort('sepsis')        # mortality among sepsis

# colors per subgroup (same color for the pair)
colors = {1:'#FFFFFF', 2:'#E41A1C', 3:'#4DAF4A', 4:'#377EB8', 5:'#4DD2D2', 6:'#E377C2'}

# ---- Panel D: two bars per subgroup (OD vs Sepsis), same color ----
x = mort_od['subgroup'].to_numpy()
w = 0.32

fig, ax = plt.subplots(figsize=(6,4))
for i, sg in enumerate(x):
    col = colors[sg]
    # left bar: organ dysfunction mortality
    ax.bar(sg - w/2, mort_od.loc[mort_od['subgroup']==sg, 'p'],
           width=w,
           yerr=np.vstack([mort_od.loc[mort_od['subgroup']==sg, 'err_low'],
                           mort_od.loc[mort_od['subgroup']==sg, 'err_high']]).astype(float),
           capsize=3, edgecolor='black', linewidth=1.0, color=col)
    # right bar: sepsis mortality
    ax.bar(sg + w/2, mort_sep.loc[mort_sep['subgroup']==sg, 'p'],
           width=w,
           yerr=np.vstack([mort_sep.loc[mort_sep['subgroup']==sg, 'err_low'],
                           mort_sep.loc[mort_sep['subgroup']==sg, 'err_high']]).astype(float),
           capsize=3, edgecolor='black', linewidth=1.0, color=col)

ax.set_xlabel('Subgroup (K=6)')
ax.set_ylabel('Percent mortality')
ax.set_xticks(x)
ax.set_ylim(0, 40)       # match paper scale
ax.set_title('D')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(True, linestyle=':', linewidth=0.7, alpha=0.6)
plt.tight_layout()
plt.show()
plt.savefig('/Users/gloriaye/Desktop/dsc180ab/25fa-dsc180a-team1/assets/fig_4_d.png', dpi=300, bbox_inches='tight')